# Jersey Order Management For Youth Hockey Teams

> This project is a jupyter notebook that allows youth hockey team managers to manage their jersey orders.

## Imports and Config

In [2]:
import base64
import os.path
import pickle

from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

import pandas as pd
import gspread

from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# Import config
from config.config import SCOPES, WORKSHEET_NAME, SENDER_EMAIL, DEFAULT_TO_EMAIL

## Auth

In [8]:
creds = None
# The file token.pickle stores the user's access and refresh tokens
if os.path.exists('config/token.pickle'):
    with open('config/token.pickle', 'rb') as token:
        creds = pickle.load(token)
        
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'config/credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('config/token.pickle', 'wb') as token:
        pickle.dump(creds, token)

## Modules

In [9]:
def read_google_sheet(sheet_name):
    """
    Read data from a Google Sheet and return it as a pandas DataFrame.
    
    Args:
        sheet_name (str): Name of the Google Sheet to read
        
    Returns:
        pandas.DataFrame: The sheet data as a DataFrame
    """
    gc = gspread.authorize(creds)

    # Open the spreadsheet
    spreadsheet = gc.open(sheet_name)

    # Get the first worksheet
    worksheet = spreadsheet.get_worksheet(0)

    # Get all values and convert to pandas DataFrame
    data = worksheet.get_all_values()
    headers = data[0]
    df = pd.DataFrame(data[1:], columns=headers)

    return df


def send_gmail(sender_email, to_email, subject, message_text):
    """
    Send an email using Gmail API.
    
    Args:
        sender_email (str): The email address of the sender
        to_email (str): The email address of the recipient
        subject (str): The subject of the email
        message_text (str): The body text of the email
        
    Returns:
        dict: The sent message object
    """
    service = build('gmail', 'v1', credentials=creds)

    # Create message
    message = MIMEMultipart()
    message['to'] = to_email
    message['from'] = sender_email
    message['subject'] = subject

    msg = MIMEText(message_text)
    message.attach(msg)

    # Encode the message
    raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode('utf-8')
    
    try:
        # Send the message
        message = service.users().messages().send(
            userId='me',
            body={'raw': raw_message}
        ).execute()
        print(f'Message Id: {message["id"]}')
        return message
    except Exception as e:
        print(f'An error occurred: {e}')
        return None

## Business Logic

In [10]:
df = read_google_sheet(WORKSHEET_NAME)
df

,Date,Person,Email,JerseySize
0,05/20/2025,Person 1,person1@example.com,S
1,05/21/2025,Person 2,person2@example.com,M
2,05/22/2025,Person 3,person3@example.com,L
3,05/23/2025,Person 4,person4@example.com,XL
4,05/24/2025,Person 5,person5@example.com,M
5,05/25/2025,Person 6,person6@example.com,S
6,05/26/2025,Person 7,person7@example.com,L
7,05/27/2025,Person 8,person8@example.com,XL
8,05/28/2025,Person 9,person9@example.com,M
9,05/29/2025,Person 10,person10@example.com,L


In [11]:
# Example usage:
send_gmail(
    sender_email=SENDER_EMAIL,
    to_email=DEFAULT_TO_EMAIL,
    subject='Jersey Order Update',
    message_text='Your jersey order has been processed.'
)

Message Id: 1974b2ebc6333860


{'id': '1974b2ebc6333860',
 'threadId': '1974b2ebc6333860',
 'labelIds': ['UNREAD', 'SENT', 'INBOX']}